In [1]:
import numpy as np
from data_loader import load_dataset, load_challenge, save_challenge
from torch import nn
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from eval_script import compute_revenue

In [2]:
class BirdCrossEntropyLoss(nn.Module):
    def __init__(self, device):
        super(BirdCrossEntropyLoss, self).__init__()
        self.rev_matrix = torch.tensor(np.array(
            [[0.05, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2],
             [-0.25,  1., -0.3, -0.1, -0.1, -0.1, -0.1],
                [-0.02, -0.1,  1., -0.1, -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3,  1., -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1,  1., -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1,  1., -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1, -0.1,  1.]])).to(device)
        self.cet = nn.CrossEntropyLoss(reduction='none').to(device)


    def forward(self, input, target):
        ce_loss = self.cet(input.squeeze(), target.squeeze())

        input_labels = torch.argmax(input, dim=2)
        target_labels = torch.argmax(target, dim=2)
        wanted = self.rev_matrix[target_labels, target_labels]
        actual = self.rev_matrix[target_labels, input_labels]
        cost = wanted - actual
        custom_loss = cost * ce_loss

        return custom_loss.mean()


In [3]:
class BirdDataset(Dataset):
    def __init__(self, birdset, is_train, device, scaler=None):
        self.device = device
        self.data = []
        self.labels = []
        for bird in birdset:
            if is_train:
                self.data += birdset[bird]['train_features']
                self.labels += birdset[bird]['train_labels']
            else:
                self.data += birdset[bird]['test_features']
                self.labels += birdset[bird]['test_labels']

        hot_matrix = np.eye(7)
        for i in range(len(self.labels)):
            self.labels[i] = hot_matrix[self.labels[i]]

        if scaler == None:
            scaler = StandardScaler()
            scaler.fit(np.concatenate(self.data))
        self.scaler = scaler

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = torch.tensor(self.scaler.transform(
            self.data[index])).to(self.device)
        y = torch.tensor(self.labels[index]).to(self.device)
        return x, y


In [20]:
def train(dataloader, model, loss_fn, optimizer, progress_steps):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if progress_steps != None and batch % progress_steps == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, show_progress):
    num_batches = len(dataloader)
    test_loss = 0
    money = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            money += compute_revenue(torch.argmax(pred, dim=2).cpu(), torch.argmax(y, dim=2).cpu())

    test_loss /= num_batches
    if show_progress:
        print(f"Avg loss: {test_loss:>8f}, Money saved: {money:.2f}$\n")
    return test_loss

def eval(model_fn, train_dataloader, test_dataloader, device, max_epochs):
    model = model_fn(None).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    loss_fn = BirdCrossEntropyLoss(device)
    lowest_test_loss = np.infty
    stop_criterion = 0
    for t in range(max_epochs):
        if t % 10 == 0:
            print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer, None)
        test_loss = test(test_dataloader, model, loss_fn, True)
        if test_loss > lowest_test_loss:
            stop_criterion += 1
        else:
            lowest_test_loss = test_loss
            stop_criterion = 0
        if stop_criterion >= 5:
            break
    print("Estimated performance:")
    test(test_dataloader, model, loss_fn, True)
    return model


In [21]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        
        return out

model_fn = lambda _: LSTMModel(548, 128, 4, 7)

In [6]:
dataset = load_dataset()

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
train_set = BirdDataset(dataset, True, device)
scaler = train_set.scaler
test_set = BirdDataset(dataset, False, device, scaler)
train_loader = DataLoader(train_set, batch_size=1)
test_loader = DataLoader(test_set, batch_size=1)

In [23]:
model = eval(model_fn, train_loader, test_loader, device, 500)

Epoch 1
-------------------------------
Avg loss: 1.272775, Money saved: 141.05$

Avg loss: 1.203765, Money saved: -40.04$

Avg loss: 1.182233, Money saved: -179.92$

Avg loss: 1.188907, Money saved: -270.40$

Avg loss: 1.117046, Money saved: -297.19$

Avg loss: 1.049587, Money saved: -335.09$

Avg loss: 0.949750, Money saved: -35.44$

Avg loss: 0.812846, Money saved: 624.03$

Avg loss: 0.685794, Money saved: 1136.29$

Avg loss: 0.616942, Money saved: 1150.66$

Epoch 11
-------------------------------
Avg loss: 0.575560, Money saved: 1207.24$

Avg loss: 0.541325, Money saved: 1357.01$

Avg loss: 0.520059, Money saved: 1415.76$

Avg loss: 0.505889, Money saved: 1421.61$

Avg loss: 0.489991, Money saved: 1476.21$

Avg loss: 0.468377, Money saved: 1511.71$

Avg loss: 0.446676, Money saved: 1708.75$

Avg loss: 0.424207, Money saved: 1897.34$

Avg loss: 0.429400, Money saved: 2007.34$

Avg loss: 0.398896, Money saved: 2209.96$

Epoch 21
-------------------------------
Avg loss: 0.380108, Mo

In [ ]:
torch.save(model.state_dict(), 'lstm')

In [ ]:
challenge = load_challenge()
print(challenge.shape)

(16, 3000, 548)


In [ ]:
model = model_fn(None)
model.load_state_dict(torch.load('lstm'))
model.eval()

model = model.to(device)
results = torch.Tensor(np.empty((challenge.shape[0], challenge.shape[1]))).to(device)
for i in range(challenge.shape[0]):
    challenge_subset = challenge[i]
    challenge_subset = scaler.transform(challenge_subset)
    challenge_subset = torch.Tensor(challenge_subset).unsqueeze(0).to(device)
    preds = model(challenge_subset).squeeze()
    pred_labels = torch.argmax(preds, dim=1)
    results[i, :] = pred_labels
results = results.int().cpu().numpy()
print(results, results.shape)
save_challenge('lstm_test', results)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 3 3 3]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] (16, 3000)
